In [2]:
from google.colab import drive
drive.mount('/content/gdrive') #, force_remount=True

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import os
os.chdir('gdrive/My Drive/Colab Notebooks')

In [ ]:
!pip install pymorphy2
!pip install pymystem3==0.1.10
!pip install rusenttokenize
!pip install razdel

In [4]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

**Задание 1.**

Реализуйте алгоритм Symspell - https://medium.com/@wolfgarbe/1000x-faster-spelling-correction-algorithm-2012-8701fcd87a5f. Он похож на алгоритм Норвига, но проще и быстрее. Там к словам в словаре применяется только одна операция - удаление символа. Чтобы найти исправление из слова тоже удаляются символы и сравниваются с теми, что хранятся в словаре. Оцените качество полученного алгоритма теми же тремя метриками.

In [5]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter, defaultdict
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)
from sklearn.metrics import classification_report, accuracy_score
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from collections import Counter

def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

def preprocess(text):
    sents = sentenize(text)
    return [normalize(sent.text) for sent in sents]

def ngrammer(tokens, n):
    ngrams = []
    tokens = [token for token in tokens]
    for i in range(0,len(tokens)-n+1):
        ngrams.append(tuple(tokens[i:i+n]))
    return ngrams

In [6]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [7]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

In [8]:
pprint(align_words(true[1], bad[1])[:5])

[('апофеозом', 'опофеозом'),
 ('дня', 'дня'),
 ('для', 'для'),
 ('меня', 'меня'),
 ('сегодня', 'сегодня')]


In [9]:
corpus = open('wiki_data.txt', encoding='utf8').read()
vocab = set(re.findall('\w+', corpus.lower()))

In [10]:
WORDS = Counter(vocab)
N = sum(WORDS.values())

In [11]:
def predict_mistaken(word, vocab):
    if word in vocab:
        return 0
    else:
        return 1

In [12]:
def P(word, N=N): 
    "Вычисляем вероятность слова"
    return WORDS[word] / N

P('солнце')

2.7114820418544366e-06

In [13]:
def edits1(word):
    "Создаем кандидатов, которые отличаются на одну букву"
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    return set(deletes)

def dict_2(vocab):
  "Pre-calculation step"
  "Generate terms with an edit distance from each dictionary term and add them together with the original term to the dictionary"
  #d = {}
  d = defaultdict(list) #иначе ломается при неизвестных словах
  for word in vocab:
    edits = edits1(word)
    for edit in edits:
      if edit in d.keys():
        d[edit].append(word)
      else:
        d[edit] = [word]
  return d

In [14]:
err_dict = dict_2(WORDS)

In [15]:
err_dict['облак']

['облаке', 'облако', 'облака']

In [16]:
def candidates(word, d = err_dict):
    "Генерируем кандидатов на исправление"
    "Generate terms with an edit distance from the input term and search them in the dictionary."
    cands = d[word] #добавим кандидатов для изначального слова
    cands.append(word) #добавим само слово
    edits = edits1(word)
    for i in edits:
      for j in d[i]:
        cands.append(j) #также добавим кандидатов для каждого варианта слова 
    return cands

def correction(word): 
    "Находим наиболее вероятное похожее слово"
    return max(candidates(word), key=P)

In [17]:
len(candidates('облак'))

16

In [18]:
correction('облак')

'облаке'

Для оценки используем будем использовать три метрики:

1) процент правильных слов;

2) процент исправленных ошибок

3) процент ошибочно исправленных правильных слов

In [22]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        # чтобы два раза не исправлять одно и тоже слово - закешируем его
        # перед тем как считать исправление проверим нет ли его в кеше
        # проверяем, что слова нет в словаре, чтобы не исправлять все слова
        if not predict_mistaken(pair[1], WORDS):
          predicted = pair[1]
        else:
          predicted = cashed.get(pair[1], correction(pair[1]))
        
        cashed[pair[1]] = predicted
        
        if predicted == pair[0]:
            correct += 1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1

print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.8643356643356643
0.2985418265541059
0.05099345354312622


**Задание 2.**

Добавьте к полученному алгоритму исправления (Symspell) триграммную модель и проверьте, улучшает ли она качество. Триграммную модель нужно вставить туда, где у вас выбирается один из нескольких кандидатов на исправление.

In [20]:
corpus_wiki = [['<start>', '<start>'] + sent + ['<end>'] for sent in preprocess(corpus)]

In [21]:
unigrams = Counter()
bigrams = Counter()
trigrams = Counter()

for sentence in corpus_wiki:
    unigrams.update(sentence)
    bigrams.update(ngrammer(sentence, 2))
    trigrams.update(ngrammer(sentence, 3))

In [24]:
mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0

for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    word_pairs = [('<start>', '<start>')] + word_pairs
    pred_sent = []
    for j in range(2, len(word_pairs)):
        pred = None
        # проверяем, что слова нет в словаре, чтобы не исправлять все слова
        if not predict_mistaken(word_pairs[j][1], WORDS):
            pred = word_pairs[j][1]
        else:
            predicted = candidates(word_pairs[j][1]) # находим кандидатов для исправления
            prev_word = word_pairs[j-2][1] + ' ' + word_pairs[j-1][1] # берем два предыдущих слова для контекста
        
            # если у нас нет в модели такой биграммы, берем первое по близости
            if prev_word not in bigrams:
                pred = predicted[0][0]
            else:
                lm_predicted = []
                for word in predicted:
                    trigram = ' '.join([prev_word, word])
                    lm_predicted.append((word, (trigrams[trigram]/bigrams[prev_word]))) 
    
                if lm_predicted:
                  pred = sorted(lm_predicted, key=lambda x: -x[1])[0][0]
        
        if pred is None:
            pred = word_pairs[j][1]
        
        if pred == word_pairs[j][0]:
            correct += 1
        else:
            mistakes.append((word_pairs[j][0], word_pairs[j][1], pred))
        total += 1
            
        if word_pairs[j][0] == word_pairs[j][1]:
            total_correct += 1
            if word_pairs[j][0] !=  pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if word_pairs[j][0] == pred:
                mistaken_fixed += 1

print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)                

0.8343963052562129
0.0026845637583892616
0.09138819020241945


Получилось так, что результаты стали хуже по всем метрикам.